# Tarea 3 MongoDB, arXiv
##### Integrantes:
Bruno Morici, ROL: 202373555-8,
Martin Aranda, ROL: 202373021-1

## Primera Etapa
Instalación de dependencias

In [6]:
# Instalamos dependencias
!pip install pymongo
!pip install tqdm


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Segunda Etapa
Conexión a la BD "arxiv_db" y creación de colección "articles"

In [3]:
from pymongo import MongoClient

# Conexión a Replica Set con preferencia al primario
client = MongoClient("mongodb://mongo1:27017,mongo2:27017,mongo3:27017/?replicaSet=rs0")

db = client["arxiv_db"]
collection = db["articles"]

try:
    server_info = client.server_info()
    print("✅ Conexión exitosa")
    print(f"Primario: {client.primary}") # El cliente retorna un primario mal detectado, es normal, pero las escrituras se realizan sobre mongo1
except Exception as e:
    print(f"❌ Error de conexión: {e}")

✅ Conexión exitosa
Primario: ('mongo3', 27017)


## Tercera Etapa
Procesamiento del dataset hacia la BD (ejecutar una sola vez, luego los datos quedan en la BD)

Observación: El dataset tiene un JSON en cada línea, no es una lista de JSON's. Recorremos cada JSON, lo parseamos y guardamos en una lista de data (demora unos 6 minutos)

In [ ]:
import json
from tqdm import tqdm # Para mostrar una barra de progreso

# Cargar y procesar el archivo JSON
print("Iniciando carga de datos a MongoDB...")

# Carga por lotes de 1000 en 1000
def load_data_batch(batch_size=1000):
    batch = []
    total_docs = 0
    
    # Primero contamos las lineas para la barra de progreso
    with open('arxiv-metadata-oai-snapshot.json', 'r', encoding='utf-8') as f:
        total_lines = sum(1 for _ in f)
    
    # Insertamos los datos en el batch
    with open('arxiv-metadata-oai-snapshot.json', 'r', encoding='utf-8') as f:
        for line in tqdm(f, total=total_lines, desc="Cargando datos"):
            batch.append(json.loads(line)) # Parseamos la linea a un JSON
            
            # Si el batch excede el maximo, lo vaciamos e insertamos los datos en la BD
            if len(batch) >= batch_size:
                collection.insert_many(batch)
                total_docs += len(batch)
                batch = []
        
        # Insertar el ultimo lote, en caso de que el ultimo lote sean menos de 1000
        if batch:
            collection.insert_many(batch)
            total_docs += len(batch)
    
    print(f"\nTotal de documentos insertados: {total_docs}")

load_data_batch() # Llamamos la funcion creada anteriormente

# 3. Verificacion final
print("\nResumen de la base de datos:")
print(f"- Colección: {collection.name}")    
print(f"- Ejemplo de documento: {collection.find_one()}")


Iniciando carga de datos a MongoDB...


Cargando datos: 100%|██████████| 2744489/2744489 [28:58<00:00, 1578.86it/s] 



Total de documentos insertados: 2744489

Resumen de la base de datos:
- Colección: articles


_OperationCancelled: operation cancelled

In [12]:
ports = [27017, 27018, 27019]
for i, port in enumerate(ports, start=1):
    client = MongoClient(f"mongodb://localhost:{port}")
    count = client["arxiv_db"]["articles"].estimated_document_count()
    print(f"📦 Documentos estimados en mongo{i}: {count}")

📦 Documentos estimados en mongo1: 2744489
📦 Documentos estimados en mongo2: 2744489
📦 Documentos estimados en mongo3: 2744489


## Cuarta etapa: Consultas

In [4]:
def buscar_software(port, nombre):
    try:
        client = MongoClient(f"mongodb://localhost:{port}", serverSelectionTimeoutMS=5000)
        db = client["arxiv_db"]
        collection = db["articles"]

        resultados = collection.find(
            {"title": {"$regex": "Software", "$options": "i"}},
            {"_id": 0, "title": 1}
        ).limit(3)  # solo para mostrar pocos

        print(f"🟢 {nombre} ({port}) - Artículos con 'Software':")
        encontrados = False
        for doc in resultados:
            print(f"   🔹 {doc['title']}")
            encontrados = True
        if not encontrados:
            print("   ⚠️ No se encontraron artículos.")
        print("-" * 50)
    except Exception as e:
        print(f"🔴 Error en {nombre} ({port}): {e}")

# Ejecutar búsqueda en los tres nodos
buscar_software(27017, "mongo1")
buscar_software(27018, "mongo2")
buscar_software(27019, "mongo3")


🟢 mongo1 (27017) - Artículos con 'Software':
   🔹 Solar Magnetic Tracking. I. Software Comparison and Recommended
  Practices
   🔹 On How Developers Test Open Source Software Systems
   🔹 Experiences of Engineering Grid-Based Medical Software
--------------------------------------------------
🟢 mongo2 (27018) - Artículos con 'Software':
   🔹 Solar Magnetic Tracking. I. Software Comparison and Recommended
  Practices
   🔹 On How Developers Test Open Source Software Systems
   🔹 Experiences of Engineering Grid-Based Medical Software
--------------------------------------------------
🟢 mongo3 (27019) - Artículos con 'Software':
   🔹 Solar Magnetic Tracking. I. Software Comparison and Recommended
  Practices
   🔹 On How Developers Test Open Source Software Systems
   🔹 Experiences of Engineering Grid-Based Medical Software
--------------------------------------------------


In [5]:
from datetime import datetime
def leer_articulos_2025(collection, client):
    """
    Lee los primeros 20 artículos cuya versión v1 fue creada en 2025.
    Muestra solo el título y la fecha de creación.
    """
    try:
        primary_info = client.primary
        print(f"📦 Leyendo datos desde: {primary_info[0]}:{primary_info[1]} (PRIMARY)")

        articulos_2025 = []

        cursor = collection.find(
            {"versions.0.created": {"$exists": True}},
            {"_id": 0, "title": 1, "versions.0.created": 1}
        )

        for doc in cursor:
            try:
                fecha_str = doc["versions"][0]["created"]
                fecha = datetime.strptime(fecha_str, "%a, %d %b %Y %H:%M:%S %Z")
                
                if fecha.year == 2025:
                    articulos_2025.append({
                        "titulo": doc.get("title", "Sin título"),
                        "fecha_creacion_v1": fecha
                    })
            except:
                continue  # ignorar si no se puede convertir la fecha

            if len(articulos_2025) == 20:
                break

        print(f"🔎 Se encontraron {len(articulos_2025)} artículos con v1 creada en 2025:")
        for articulo in articulos_2025:
            print(articulo)

        return articulos_2025

    except Exception as e:
        print(f"❌ Error al leer artículos de 2025: {e}")
        return []

leer_articulos_2025(collection, client)

📦 Leyendo datos desde: mongo3:27017 (PRIMARY)
🔎 Se encontraron 0 artículos con v1 creada en 2025:


[]